In [1]:
import re
import pandas as pd
from collections import Counter
import os

In [2]:
def accented_syllable(line):
  line_syllable_order = []
  vowels_in_str = re.findall(r"[АИЭУЕЁЫОЮЯаиэуеёыоюя]'?", line)
  for idx, accent in enumerate(vowels_in_str):
    right_idx = idx + 1
    if accent.endswith("\'"):
      line_syllable_order.append(int(right_idx))
  return line_syllable_order

In [3]:
def intervals(line):
  intervals = []
  i = 1
  x = i - 1
  if len(line) == 1:
      intervals.append(0)
  else:
    while i < len(line):
      interval = line[i] - line[x]
      intervals.append(interval)
      i += 1
      x += 1
  return intervals

In [5]:
def meter_in_lines(line_inter, line_acc):
  if len(line_acc) == 1:
    meter = "exception"
  else:
    if all(map(lambda x: x % 2 == 0, line_inter)) == True and all(map(lambda x: x % 3 == 0, line_inter)) == True:
      meter = "exception"
    elif all(map(lambda x: x % 2 == 0, line_inter)) == True:
      if all(map(lambda x: x % 2 == 0, line_acc)) == True:
        meter = "iambus"
      elif all(map(lambda x: x % 2 != 0, line_acc)) == True:
        meter = "trochee"
      else:
        meter = "other"
    elif all(map(lambda x: x % 3 == 0, line_inter)) == True:
      if all(map(lambda x: (x - 1) % 3 == 0, line_acc)) == True:
        meter = "dactyl"
      elif all(map(lambda x: (x - 2) % 3 == 0, line_acc)) == True:
        meter = "amphibrach"
      elif all(map(lambda x: (x - 3) % 3 == 0, line_acc)) == True:
        meter = "anapaest"
      else:
        meter = "other"
    else:
      meter = "other"
  return meter

In [6]:
def other_meter(line_inter, line_acc):
  if len(list(set(line_inter))) == 3 and 1 in line_inter and 2 in line_inter and 3 in line_inter:
    if (line_inter.count(3) != 1 and line_inter.count(3) > line_inter.count(2) and line_inter.count(3) > line_inter.count(1)) or (line_inter.count(3) == line_inter.count(2) == line_inter.count(1) == 1):
      inter_one = [i for i in range(len(line_inter)) if line_inter[i] == 1]
      inter_two = [i for i in range(len(line_inter)) if line_inter[i] == 2]
      if len(inter_one) != len(inter_two):
        meter_n = "other"
      else:
        i = 0
        pre_meter = []
        while i < len(inter_one):
          if abs(inter_one[i] - inter_two[i]) != 1:
            pre_meter.append("other")
            break
          else:
            i += 1
            continue
        for accent in line_acc:
          if (accent - 1) % 3 == 0:
            pre_meter.append("dactyl")
          elif (accent - 2) % 3 == 0:
            pre_meter.append("amphibrach")
          elif (accent - 3) % 3 == 0:
            pre_meter.append("anapaest")
          else:
            pre_meter.append("other")
        if "other" in pre_meter:
          meter_n = "other"
        else:
          stats_pre_meter = Counter(pre_meter)
          meter_n = max(stats_pre_meter, key = stats_pre_meter.get)
    else:
      meter_n = "other"
  elif len(list(set(line_inter))) == 2 and 3 in line_inter:
    if (2 in line_inter and line_inter.count(3) > line_inter.count(2) == 1) or (1 in line_inter and line_inter.count(3) > line_inter.count(1) == 1):
      if (1 in line_inter and line_inter.index(1) != 0) or (2 in line_inter and line_inter.index(2) != 0):
        meter_n = "other"
      else:
        pre_meter = []
        for accent in line_acc:
          if (accent - 1) % 3 == 0:
            pre_meter.append("dactyl")
          elif (accent - 2) % 3 == 0:
            pre_meter.append("amphibrach")
          elif (accent - 3) % 3 == 0:
            pre_meter.append("anapaest")
          else:
            pre_meter.append("other")
        stats_pre_meter = Counter(pre_meter)
        meter_n = max(stats_pre_meter, key = stats_pre_meter.get)
    else:
      meter_n = "other"
  else:
    meter_n = "other"
  return meter_n

In [7]:
def simple_all_meters(list_m):
  if "iambus" in list_m:
    final_meter = "iambus"
  elif "trochee" in list_m:
    final_meter = "trochee"
  elif "dactyl" in list_m:
    final_meter = "dactyl"
  elif "amphibrach" in list_m:
    final_meter = "amphibrach"
  elif "anapaest" in list_m:
    final_meter = "anapaest"
  else:
    final_meter = "other"
  return final_meter

In [8]:
def complex_all_meters(list_m):
  for_iam = list(range(2, 20, 2))
  for_tr = list(range(1, 20, 2))
  for_dac = list(range(1, 20, 3))
  for_amp = list(range(2, 20, 3))
  for_an = list(range(3, 20, 3))

  if "iambus" in list_m:
    pre_final_meter = []
    for meter in list_m:
      i = 0
      while i < len(list_m):
        if list_m[i] == "iambus" or list_m[i] in for_iam:
          pre_final_meter.append("iambus")
        else:
          pre_final_meter.append("other")
        i += 1
    if len(list(set(pre_final_meter))) == 1 and "iambus" in pre_final_meter:
      final_meter = "iambus"
    else:
      final_meter = "other"
  elif "trochee" in list_m:
    pre_final_meter = []
    for meter in list_m:
      i = 0
      while i < len(list_m):
        if list_m[i] == "trochee" or list_m[i] in for_tr:
          pre_final_meter.append("trochee")
        else:
          pre_final_meter.append("other")
        i += 1
    if len(list(set(pre_final_meter))) == 1 and "trochee" in pre_final_meter:
      final_meter = "trochee"
    else:
      final_meter = "other"
  elif "dactyl" in list_m:
    pre_final_meter = []
    for meter in list_m:
      i = 0
      while i < len(list_m):
        if list_m[i] == "dactyl" or list_m[i] in for_dac:
          pre_final_meter.append("dactyl")
        else:
          pre_final_meter.append("other")
        i += 1
    if len(list(set(pre_final_meter))) == 1 and "dactyl" in pre_final_meter:
      final_meter = "dactyl"
    else:
      final_meter = "other"
  elif "amphibrach" in list_m:
    pre_final_meter = []
    for meter in list_m:
      i = 0
      while i < len(list_m):
        if list_m[i] == "amphibrach" or list_m[i] in for_amp:
          pre_final_meter.append("amphibrach")
        else:
          pre_final_meter.append("other")
        i += 1
    if len(list(set(pre_final_meter))) == 1 and "amphibrach" in pre_final_meter:
      final_meter = "amphibrach"
    else:
      final_meter = "other"
  elif "anapaest" in list_m:
    pre_final_meter = []
    for meter in list_m:
      i = 0
      while i < len(list_m):
        if list_m[i] == "anapaest" or list_m[i] in for_an:
          pre_final_meter.append("anapaest")
        else:
          pre_final_meter.append("other")
        i += 1
    if len(list(set(pre_final_meter))) == 1 and "anapaest" in pre_final_meter:
      final_meter = "anapaest"
    else:
      final_meter = "other"
  else:
    final_meter = "other"
  return final_meter

In [15]:
poem_whole_meter = []
poem_name = []
poem_accents = []

voz = os.listdir()
for n in voz:
  if n.endswith(".txt") == False:
    voz.remove(n)

for n in voz:
  with open(n, encoding="utf-8") as poem:
    lines = [x.strip() for x in poem]
    all_meters = []
    all_accents = []
    for line in lines:
      if "'" not in line:
        continue
      else:
        line_syllable_order = accented_syllable(line)
      if line_syllable_order == []:
        continue
      else:
        all_accents.append(line_syllable_order)
      intervals_line = intervals(line_syllable_order)
      pre_meter_line = meter_in_lines(intervals_line, line_syllable_order)
      if pre_meter_line == "other":
        meter_line = other_meter(intervals_line, line_syllable_order)
      elif pre_meter_line == "exception":
        meter_line = line_syllable_order[0]
      else:
        meter_line = pre_meter_line
      all_meters.append(meter_line)
    unique_all_meters = list(set(all_meters))
    if len(unique_all_meters) == 1:
      poem_meter = simple_all_meters(all_meters)
    else:
      poem_meter = complex_all_meters(all_meters)
    just_name, no_need_1, no_need_2 = n.rsplit(".", maxsplit=2)
    poem_whole_meter.append(poem_meter)
    poem_name.append(just_name)
    poem_accents.append(all_accents)

  accents_df = pd.DataFrame({"name": poem_name, "meter_poem": poem_whole_meter, "accents": poem_accents})
  accents_df.to_csv("METRES.csv")